In [ ]:
!pip install -q unsloth datasets accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.8/306.8 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.9/511.9 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.4/176.4 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.1/117.1 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.7/155.7 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from unsloth import FastLanguageModel
import torch

model_name = "unsloth/llama-3.1-8b-bnb-4bit"  # QLoRA-ready

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=2048,
    dtype=None,          # Auto
    load_in_4bit=True,   # QLoRA mode
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.8.4: Fast Llama patching. Transformers: 4.55.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


<string>:37: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset

# Load dataset
dataset = load_dataset("jaimevera1107/moderation-topics-dataset")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Topics.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/1704 [00:00<?, ? examples/s]

In [ ]:
# Print information about the dataset
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['GeneralTopic', 'MetaTopic', 'Topic', 'SubTopic', 'SpecificTopic'],
        num_rows: 1704
    })
})


In [ ]:
# Print the first few examples from the training split
dataset["train"][:5]

{'GeneralTopic': ['Hate, Slur, Bullying, and Harrasment',
  'Inorganic Content',
  'Regulated or Illegal Goods',
  'Minors Content',
  'Inorganic Content'],
 'MetaTopic': ['Bullying', 'Inorganic', 'Drugs', 'Pedophilia', 'Ads'],
 'Topic': ['Bullying Behavior or Statements, Cyberbullying, Harrasment, Language Targeting',
  'Inorganic Interaction Or Content, No Interaction, Low Quality or Viewing',
  'Drugs And Substances Use, Paraphernalia, Controlled Substances, Cannabis, Psychoactive',
  'Pedophilia, Grooming Behavior, Luring, Predators',
  'Advertisement, Ads, Commercial, Platforms, Promotional, Sales Content'],
 'SubTopic': ['Bullying', 'Reproduced', 'Drugs', 'Pedophilia', 'Promotional'],
 'SpecificTopic': ['Abusive Language',
  'Account Farming',
  'Addiction Or Substance Use Support',
  'Admission Of Romantic Pedophilic Relationship',
  'Ads']}

In [ ]:
len(dataset["train"])

1704

In [ ]:
dataset["train"][0]

{'GeneralTopic': 'Hate, Slur, Bullying, and Harrasment',
 'MetaTopic': 'Bullying',
 'Topic': 'Bullying Behavior or Statements, Cyberbullying, Harrasment, Language Targeting',
 'SubTopic': 'Bullying',
 'SpecificTopic': 'Abusive Language'}

['Hate, Slur, Bullying, and Harrasment',
 'Inorganic Content',
 'Regulated or Illegal Goods',
 'Minors Content',
 'Inorganic Content',
 'Explicit or Graphic Content',
 'Explicit or Graphic Content',
 'Adult Content',
 'Adult Content',
 'Adult Content']

In [ ]:
# Keep only "suggestive" and "safe"
def filter_suggestive(example):
    return example["category"] in ["Suggestive", "Safe"]

dataset = dataset.filter(filter_suggestive)

# Create instruction-response pairs
def format_example(example):
    return {
        "instruction": "Classify whether the text is suggestive or safe.",
        "input": example["text"],
        "output": example["category"]
    }

dataset = dataset.map(format_example)

train_data = dataset["train"]
val_data   = dataset["test"]

In [ ]:
# Keep only "suggestive" and "safe"
def filter_suggestive(example):
    return example["category"] in ["Suggestive", "Safe"]

dataset = dataset.filter(filter_suggestive)

# Create instruction-response pairs
def format_example(example):
    return {
        "instruction": "Classify whether the text is suggestive or safe.",
        "input": example["text"],
        "output": example["category"]
    }

dataset = dataset.map(format_example)

train_data = dataset["train"]
val_data   = dataset["test"]

In [ ]:
dataset=load_dataset("Morteza-Shahrabi-Farahani/Detecting-toxic-comments")

README.md:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

data/raw/initial-files/train.csv:   0%|          | 0.00/68.8M [00:00<?, ?B/s]

data/raw/initial-files/test.csv:   0%|          | 0.00/60.4M [00:00<?, ?B/s]

test_labels.csv: 0.00B [00:00, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'],
        num_rows: 159571
    })
    test: Dataset({
        features: ['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'],
        num_rows: 306328
    })
})


In [ ]:
print(dataset['train'][:5])

{'id': ['0000997932d777bf', '000103f0d9cfb60f', '000113f07ec002fd', '0001b41b1c6bb37e', '0001d958c54c6e35'], 'comment_text': ["Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27", "D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11, 2016 (UTC)", "Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.", '"\nMore\nI can\'t make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of ""types of accidents""  -I think the references may need tidying so that they are all in the exact same format ie 

In [ ]:
import pandas as pd

df = pd.DataFrame(dataset['train'])

False

In [ ]:
toxic_comments_df = df[df['toxic'] == 1]
display(toxic_comments_df)